In [119]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import time
import datetime

In [114]:
def import_district_file(file_location):
    district_df = pd.read_csv(file_location)
    return district_df

def get_district_dict():
    district_df = import_district_file('C:/Users/ballinj/housing/london_district_codes.csv')
    district_dict = dict(zip(list(district_df['district']), list(district_df['code'])))
    return district_dict

def get_borough_dict():
    df = pd.read_csv('london_borough_list.csv')
    borough_dict = dict(zip(list(df['borough']),list(df['code'])))
    return borough_dict

def get_no_results(soup):
    no_results = soup.find('span', attrs={'class':'searchHeader-resultCount'}).text.strip()
    return no_results

def get_soup(index, region):
    district_id = district_dict[region]
    cert = "C:/Users/ballinj/housing/ca-certificates.crt"
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36'}
    url = 'https://www.rightmove.co.uk/property-for-sale/find.html'
    params = {'minBedrooms':2,
              'propertyTypes':'detached%2Cflat%2Csemi-detached%2Cterraced',
              'keywords':'',                  
              'dontShow':'retirement%2CsharedOwnership',
              'channel':'BUY',
              'secondaryDisplayPropertyType':'housesandflats',
              'index': str(index), 
              'retirement':'false',
              'includeSSTC':'false',
              'partBuyPartRent':'false',
              'sortType':2,
              'minPrice':200000,
              'viewType':'list',
              'maxPrice':450000,
              'radius':0.0,
              'locationIdentifier':'OUTCODE%' + str(district_id)[str(district_id).find('%')+1:]}
    params_string = "&".join("%s=%s" % (k,v) for k,v in params.items())
    r = requests.get(url, params=params_string)
    c = r.content    
    soup = BeautifulSoup(c, 'html.parser') 
    return soup

def get_json(index, region):
    district_id = district_dict[region]
    cert = "C:/Users/ballinj/housing/ca-certificates.crt"
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/32.0.1667.0 Safari/537.36'}
    url = 'https://www.rightmove.co.uk/property-for-sale/map.html'
    params = {'minBedrooms':2,
              'propertyTypes':'detached%2Cflat%2Csemi-detached%2Cterraced',
              'keywords':'',                  
              'dontShow':'retirement%2CsharedOwnership',
              'channel':'BUY',
              'secondaryDisplayPropertyType':'housesandflats',
              'index': str(index), 
              'retirement':'false',
              'includeSSTC':'false',
              'partBuyPartRent':'false',
              'sortType':2,
              'minPrice':200000,
              'viewType':'map',
              'maxPrice':450000,
              'radius':0.0,
              'locationIdentifier':'OUTCODE%' + str(district_id)[str(district_id).find('%')+1:]}
    params_string = "&".join("%s=%s" % (k,v) for k,v in params.items())
    r = requests.get(url, params=params_string)
    c = r.content
    soup = BeautifulSoup(c, 'html.parser')
    scripts = soup.findAll('script')
    script_list = [script if 'window.jsonModel' in str(script) else '' for script in scripts]
    script_list = [script for script in script_list if script != '']
    script = str(script_list[0])
    script = script[script.find('{'):script.rfind('}')+1]
    properties_json = json.loads(script)
    print('json retrieved')
    return properties_json

def format_data_soup(soup, region):
    no_results = get_no_results(soup)
    print(no_results + ' results found')
    print('obtaining soup...')
    index_array = np.arange(0,int(no_results)+24,24).tolist()
    listing_ids, links, property_types, addresses, prices, prices_per_week, featured_properties = [],[],[],[],[],[],[]
    added_reduced_array, letting_agent_name, letting_agent_number, num_pictures = [],[],[],[]
    for index in index_array:
        soup = get_soup(index, region)
        main_data = soup.find('div', attrs={'class':'main'})
        search_results = soup.find('div', attrs={'class':'l-searchResults'})
        ids = search_results.findAll('a', attrs={'class':'propertyCard-anchor'})#['id']
        for id in ids:
            listing_ids.append(id['id'][4:])
        listing_data = search_results.findAll('div', attrs={'class':'propertyCard-wrapper'})
        for listing in listing_data:
            featured_properties.append(listing.find('div', attrs={'class':'propertyCard-moreInfoFeaturedTitle'}).text.strip())

            details = listing.find('div', attrs={'class':'propertyCard-details'})
            addresses.append(listing.find('address').text.strip())
            property_types.append(listing.find('h2').text.strip())
            links.append('https://www.rightmove.co.uk/' + details.find('a')['href'])

            pricing = listing.find('div', attrs={'class':'propertyCard-price'})
            prices.append(pricing.find('div', attrs={'class':'propertyCard-priceValue'}).text.strip())
            added_reduced_array.append(listing.find('div', attrs={'class':'propertyCard-branchSummary'}).find('span', attrs={'class':'propertyCard-branchSummary-addedOrReduced'}).text.strip())
            estate_agent = listing.find('div', attrs={'class':'propertyCard-branchSummary'}).find('span', attrs={'class':'propertyCard-branchSummary-branchName'}).text.strip()
            estate_agent = estate_agent[estate_agent.find('by')+3:].strip()
            letting_agent_name.append(estate_agent)
            letting_agent_number.append(listing.find('div', attrs={'class':'propertyCard-contacts'}).find('a', attrs={'class':'propertyCard-contactsPhoneNumber'}).text.strip())
            meta_data = listing.find('div', attrs={'class':'propertyCard-moreInfoMeta'})
            num_pictures.append(meta_data.find('span', attrs={'class':'propertyCard-moreInfoNumber'}).text.strip())
        time.sleep(2)
    listing_df = pd.DataFrame(listing_ids, columns=['listing_id'])
    listing_df['address'] = addresses
    listing_df['property_type'] = property_types
    listing_df['property_link'] = links
    listing_df['price'] = prices
    listing_df['added/reduced_date'] = added_reduced_array
    listing_df['agent_name'] = letting_agent_name
    listing_df['agent_number'] = letting_agent_number
    listing_df['no_pictures'] = num_pictures
    listing_df['featured_property'] = featured_properties
    listing_df = listing_df[~listing_df['property_type'].str.contains('share')]
    listing_df = listing_df[~listing_df['property_type'].str.contains('Parking')]
    listing_df = listing_df[listing_df['address']!=""]
    listing_df = listing_df[listing_df['featured_property']==""]
    listing_df = listing_df.reset_index(drop=True)
    print('soup formatted')
    return listing_df

def format_data_json(properties_json):
    property_id, coordinates = [],[]
    for row in properties_json['properties']:
        property_id.append(str(row['id']))
        coordinates.append([row['location']['latitude'], row['location']['longitude']])
    coordinates_dict = dict(zip(property_id, coordinates))
    print('json formatted')
    return coordinates_dict

def housing_data_merge(housing_soup_data, housing_json_data):
    latitudes, longitudes = [],[]
    housing_soup_data = housing_soup_data[housing_soup_data['listing_id'].isin(list(housing_json_data.keys()))]
    for id in housing_soup_data['listing_id'].tolist():
        latitudes.append(housing_json_data[str(id)][0])
        longitudes.append(housing_json_data[str(id)][1])
    housing_soup_data['latitude'] = latitudes
    housing_soup_data['longitude'] = longitudes
    return housing_soup_data

district_dict = get_district_dict()

district_list = import_district_file('C:/Users/ballinj/housing/london_district_codes.csv')['district'].tolist()
district_list.remove('CR0')
housing_data_total = pd.DataFrame()
i = 1
for district in district_list:
    print(district)
    housing_soup = get_soup(index=0, region=district)
    housing_soup_data = format_data_soup(housing_soup, region=district)
    housing_json = get_json(index=0, region=district)
    housing_json_data = format_data_json(housing_json)
    housing_data_merge_df = housing_data_merge(housing_soup_data, housing_json_data)
    housing_data_merge_df['district'] = district
    housing_data_total = pd.concat([housing_data_total,housing_data_merge_df], ignore_index=True)
    time.sleep(5)
    print(district + ' complete')
    print(str(i) + ' out of ' + str(len(district_list)))
    print('\n')
    i += 1

BR1
141 results found
obtaining soup...
soup formatted
json retrieved
json formatted
BR1 complete
1 out of 297


BR2
140 results found
obtaining soup...
soup formatted
json retrieved
json formatted
BR2 complete
2 out of 297


BR3
97 results found
obtaining soup...
soup formatted
json retrieved
json formatted
BR3 complete
3 out of 297


BR4
23 results found
obtaining soup...
soup formatted
json retrieved
json formatted
BR4 complete
4 out of 297


BR5
143 results found
obtaining soup...
soup formatted
json retrieved
json formatted
BR5 complete
5 out of 297


BR6
113 results found
obtaining soup...
soup formatted
json retrieved
json formatted
BR6 complete
6 out of 297


BR7
50 results found
obtaining soup...
soup formatted
json retrieved
json formatted
BR7 complete
7 out of 297


BR8
71 results found
obtaining soup...
soup formatted
json retrieved
json formatted
BR8 complete
8 out of 297


CM13
58 results found
obtaining soup...
soup formatted
json retrieved
json formatted
CM13 complete
9

C:\Users\ballinj\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ballinj\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:147: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\ballinj\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

E14 complete
33 out of 297


E15
108 results found
obtaining soup...
soup formatted
json retrieved
json formatted
E15 complete
34 out of 297


E16
120 results found
obtaining soup...
soup formatted
json retrieved
json formatted
E16 complete
35 out of 297


E17
139 results found
obtaining soup...
soup formatted
json retrieved
json formatted
E17 complete
36 out of 297


E18
35 results found
obtaining soup...
soup formatted
json retrieved
json formatted
E18 complete
37 out of 297


E1W
14 results found
obtaining soup...
soup formatted
json retrieved
json formatted
E1W complete
38 out of 297


E2
36 results found
obtaining soup...
soup formatted
json retrieved
json formatted
E2 complete
39 out of 297


E3
115 results found
obtaining soup...
soup formatted
json retrieved
json formatted
E3 complete
40 out of 297


E4
108 results found
obtaining soup...
soup formatted
json retrieved
json formatted
E4 complete
41 out of 297


E5
29 results found
obtaining soup...
soup formatted
json retrieved


38 results found
obtaining soup...
soup formatted
json retrieved
json formatted
KT5 complete
106 out of 297


KT6
79 results found
obtaining soup...
soup formatted
json retrieved
json formatted
KT6 complete
107 out of 297


KT7
15 results found
obtaining soup...
soup formatted
json retrieved
json formatted
KT7 complete
108 out of 297


KT8
43 results found
obtaining soup...
soup formatted
json retrieved
json formatted
KT8 complete
109 out of 297


KT9
63 results found
obtaining soup...
soup formatted
json retrieved
json formatted
KT9 complete
110 out of 297


N1
22 results found
obtaining soup...
soup formatted
json retrieved
json formatted
N1 complete
111 out of 297


N10
42 results found
obtaining soup...
soup formatted
json retrieved
json formatted
N10 complete
112 out of 297


N11
57 results found
obtaining soup...
soup formatted
json retrieved
json formatted
N11 complete
113 out of 297


N12
63 results found
obtaining soup...
soup formatted
json retrieved
json formatted
N12 comple

SE24 complete
177 out of 297


SE25
187 results found
obtaining soup...
soup formatted
json retrieved
json formatted
SE25 complete
178 out of 297


SE26
77 results found
obtaining soup...
soup formatted
json retrieved
json formatted
SE26 complete
179 out of 297


SE27
50 results found
obtaining soup...
soup formatted
json retrieved
json formatted
SE27 complete
180 out of 297


SE28
91 results found
obtaining soup...
soup formatted
json retrieved
json formatted
SE28 complete
181 out of 297


SE3
54 results found
obtaining soup...
soup formatted
json retrieved
json formatted
SE3 complete
182 out of 297


SE4
27 results found
obtaining soup...
soup formatted
json retrieved
json formatted
SE4 complete
183 out of 297


SE5
50 results found
obtaining soup...
soup formatted
json retrieved
json formatted
SE5 complete
184 out of 297


SE6
111 results found
obtaining soup...
soup formatted
json retrieved
json formatted
SE6 complete
185 out of 297


SE7
36 results found
obtaining soup...
soup for

127 results found
obtaining soup...
soup formatted
json retrieved
json formatted
UB4 complete
249 out of 297


UB5
140 results found
obtaining soup...
soup formatted
json retrieved
json formatted
UB5 complete
250 out of 297


UB6
97 results found
obtaining soup...
soup formatted
json retrieved
json formatted
UB6 complete
251 out of 297


UB7
110 results found
obtaining soup...
soup formatted
json retrieved
json formatted
UB7 complete
252 out of 297


UB8
100 results found
obtaining soup...
soup formatted
json retrieved
json formatted
UB8 complete
253 out of 297


UB9
40 results found
obtaining soup...
soup formatted
json retrieved
json formatted
UB9 complete
254 out of 297


W10
3 results found
obtaining soup...
soup formatted
json retrieved
json formatted
W10 complete
255 out of 297


W11
3 results found
obtaining soup...
soup formatted
json retrieved
json formatted
W11 complete
256 out of 297


W12
18 results found
obtaining soup...
soup formatted
json retrieved
json formatted
W12 co

In [138]:
housing_data_total = pd.read_csv('properties_by_district_2019-02-27.csv')
housing_data_total['price'] = housing_data_total['price'].str.replace('£','')
# housing_data_total = housing_data_total[['listing_id','district','address','price','no_rooms','property_types','property_link','added/reduced','added/reduced_dates','scrape_date','no_pictures','latitude','longitude','agent_name','agent_number']]
# housing_data_total['scrape_date'] = str(datetime.datetime.today().date())
# room_df = pd.DataFrame(housing_data_total['property_type'].str.split(' bedroom ',1).tolist(),
#                                    columns = ['no_rooms','property_types'])
# reduced_df = pd.DataFrame(housing_data_total['added/reduced_date'].str.split(' on ',1).tolist(),
#                                    columns = ['added/reduced','added/reduced_dates'])
# housing_data_total = pd.concat([housing_data_total,room_df], axis=1)
# housing_data_total = pd.concat([housing_data_total,reduced_df], axis=1)
# housing_data_total['property_type'] = housing_data_total['property_type'].str.replace(' for sale','')
# housing_data_total = housing_data_total[['listing_id','district','address','price','no_rooms','property_types','property_link','added/reduced','added/reduced_dates','scrape_date','no_pictures','latitude','longitude','agent_name','agent_number']]
housing_data_total.to_csv('properties_by_district_2019-02-27.csv', index=False)

In [ ]:
# look at houses added yesterday/today